In [88]:
import os
import pandas as pd

In [100]:
base_dir = 'C:/Users/prayme/Documents/카카오톡 받은 파일/preprocessing_log'
log_dir = os.path.join(base_dir, 'log')
log_result_dir = os.path.join(log_dir, '_')
attendance_dir = os.path.join(base_dir, 'attendance')
encoding_type = 'utf-8-sig'

In [101]:
def preprocessing_log():
    result_dir = os.path.join(log_dir, '_')
    # 사전에 넘겨받은 로그 파일을 수작업으로 전처리 해야한다.
    # 1. 출석부를 참고하여 이름 정형화 (딱 이름만 존재하도록)
    # 2. column 이름 고정 -->> 이름, 참가시간, 나간시간, 기간(분)
    # 3. csv-utf8 형식으로 저장
    
    # 결과 디렉토리 제거 후 리턴
    work_dir = os.listdir(log_dir)
    work_dir.remove('_')
#     print([f for os.listdir(log_dir) in x x != '_'])
    for file in work_dir:
        print(file)
        # csv 불러오기
        log = pd.read_csv(os.path.join(log_dir, file), encoding=encoding_type)
        
        # 참가시간과 나간 시간을 합칠 때를 위해 정렬
        log.sort_values(by=['이름', '참가시간'])
        
        # 참가 시간과 나간 시간 합치기
        # try - except를 활용해서 데이터가 존재하지 않으면 생성한다.
        DICT = {}
        for idx, row in log.iterrows():
            try:
                DICT[row['이름']][1] = row['나간시간']
                DICT[row['이름']][2] += row['기간(분)']
            except:
                DICT[row['이름']] = [row['참가시간'], row['나간시간'], row['기간(분)']]
        
        
        # dataframe 저장
        df = pd.DataFrame.from_dict(DICT)
        # column과 row 뒤집기
        df = df.T
        # index 교체하기 / column 이름 변경하기
        df['이름']= df.index
        df.index = [i for i in range(len(DICT))]
        df.rename(columns = {0: '참가시간', 1: '나간시간', 2:'기간(분)'}, inplace=True)
        
        # df 저장
        df.to_csv(os.path.join(log_result_dir, '_' + file), encoding=encoding_type)

In [115]:
def matching_data():
    for file in os.listdir(log_result_dir):
        # 재료 불러오기
        # 출석부의 이름은 1.csv로 만들어줘야함
        # 출석부 column 전처리 필요함 ['학과', '학번', '학년', '이름', '성별']
        log = pd.read_csv(os.path.join(log_result_dir, file), encoding=encoding_type)
        attendance = pd.read_csv(os.path.join(attendance_dir, '1.csv'), encoding=encoding_type)
        
        # 데이터 매칭
        result = []
        for idx, row in log.iterrows():
            rows = attendance[row['이름'] == attendance['이름']].loc[:, ['학과', '학번', '학년', '이름', '성별']].values.tolist()
            if rows:
                csv_row = row.values.tolist()[2:]
                result.append(rows[0] + csv_row)
                
        df = pd.DataFrame(result)
        df.to_csv(os.path.join(base_dir, 'result_' + file), encoding=encoding_type)

In [116]:
preprocessing_log()
matching_data()

로그기록_201112.csv


TypeError: pop() missing 1 required positional argument: 'item'